In [1]:
import re
import numpy as np
import nibabel as nib
from pathlib import Path
from statsmodels.stats.multitest import multipletests

from src import custom_plotting as cp
from nilearn import surface, datasets
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def correct(ps_, rs_, p_crit=5e-2):
    sig_bool, ps_corrected, _, _ = multipletests(ps_, alpha=p_crit, method='fdr_bh')
    indices = np.where(sig_bool)[0]
    return sig_bool, np.abs(rs_[indices]).min()

def filter_r(rs, ps):
    # remove nan
    indices = np.isnan(rs)
    rs[indices] = 0
    ps[indices] = 1

    # correct
    ps, threshold = correct(ps, rs)
    ps = np.invert(ps)
    indices = np.where(ps)[0]
    rs[indices] = 0.
    rs_mask = np.copy(rs)
    rs_mask[rs != 0.] = 1.
    return rs, rs_mask, threshold

In [3]:
process = 'PlotVoxelBetas'
sid = '04'
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis'
data_dir = f'{top_dir}/data/interim'
out_dir = f'{top_dir}/reports/figures/{process}/sub-{sid}'
Path(out_dir).mkdir(parents=True, exist_ok=True)
fsaverage = datasets.fetch_surf_fsaverage()
roi_parcel = None
cmap = sns.color_palette("viridis", as_cmap=True)

In [4]:
mask_im = nib.load(f'{data_dir}/Reliability/sub-{sid}_set-test_stat-rho_statmap.nii.gz')
mask = np.load(f'{data_dir}/Reliability/sub-{sid}_set-test_reliability-mask.npy').astype('bool')

In [5]:
betas_name = f'{data_dir}/VoxelEncoding/sub-{sid}_model-full_predict-JointactionCommunicationCooperationDominanceIntimacyValenceArousal_control-conv2_pca-False_betas.npy'
betas = np.load(betas_name)
ps = np.load(f'{data_dir}/VoxelPermutation/sub-{sid}_model-full_predict-JointactionCommunicationCooperationDominanceIntimacyValenceArousal_control-conv2_pca-False_ps.npy')

In [6]:
features = betas_name.split('predict')[-1].split('_')[0].replace('-', '')
features = re.findall('[A-Z][^A-Z]*', features)

In [7]:
for i_feature, feature in enumerate(features):
    volume = betas[:,i_feature]
    ps_bool, _, _, _ = multipletests(ps)
    ps_bool = np.invert(ps_bool)
    volume[ps_bool] = 0.

    volume = cp.mkNifti(volume, mask, mask_im)
    texture = {'left': surface.vol_to_surf(volume, fsaverage['pial_left'],
                                           interpolation='nearest'),
               'right': surface.vol_to_surf(volume, fsaverage['pial_right'],
                                            interpolation='nearest')}

    vmax = np.nanmax(texture['right'])
    threshold = np.min(np.abs(texture['right'][texture['right'] != 0]))

    cp.plot_betas(fsaverage, texture,
                  roi=roi_parcel,
                  cmap=cmap,
                  modes=['lateral', 'ventral'],
                  threshold=threshold,
                  vmax=vmax,
                  output_file=f'{out_dir}/{feature}.png')

/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))
/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))
/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))
/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))
/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))
/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by ze